In [ ]:
import joblib
import warnings
from utils import *
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import product
from tqdm import tqdm
from sklearn import preprocessing
from geopy.distance import geodesic

import torch
from torch_geometric.data import Data

warnings.filterwarnings("ignore")

dataset_period = [datetime.strptime('2013-07-01', '%Y-%m-%d'), datetime.strptime('2017-10-01', '%Y-%m-%d')]
test_period = [datetime.strptime('2017-10-01', '%Y-%m-%d') - timedelta(days=80), datetime.strptime('2017-10-01', '%Y-%m-%d')]
valid_period = [test_period[0] - timedelta(days=40), test_period[0]]
train_period = [dataset_period[0], valid_period[0]]
predict_time = "H"

In [ ]:
features = pd.read_hdf('process_data_201306_201709/features.h5', key='features')
station_info = pd.read_hdf('process_data_201306_201709/features.h5', key="info")
alive_df = pd.read_hdf('process_data_201306_201709/features.h5', key='alive')
df_raw = pd.read_hdf('process_data_201306_201709/features.h5', key="raw")
df_raw['tripduration'] = (df_raw.stoptime - df_raw.starttime).dt.seconds
df_raw.query('starttime >= @train_period[0] & stoptime < @train_period[1]', inplace=True)

# Distance Graph

In [ ]:
%%time
distance_graph = (
    station_info[["stationid", "lat", "lon"]]
    .assign(merge_key=1)
)
distance_graph = distance_graph.merge(distance_graph, on="merge_key").drop(
    "merge_key", axis=1
)
distance_graph["distance"] = distance_graph.apply(
    lambda x: geodesic((x.lat_x, x.lon_x), (x.lat_y, x.lon_y)).meters, axis=1
)
distance_graph = distance_graph.pivot(
    index="stationid_x", columns="stationid_y", values="distance"
)
distance_graph = distance_graph ** -1

for i in range(len(distance_graph)):
    distance_graph.iloc[i,i] = 0

distance_graph = distance_graph.replace([np.inf, -np.inf], np.nan)
distance_graph = distance_graph.fillna(0)

# Rent Second Graph

In [ ]:
%%time
second_graph = (
    df_raw[["startstationid", "endstationid", "tripduration"]]
    .groupby(["startstationid", "endstationid"], as_index=False)
    .agg({"tripduration": np.median})
)
second_graph = second_graph.pivot(
    index="startstationid", columns="endstationid", values="tripduration"
)
second_graph = second_graph ** -1
for i in range(len(second_graph)):
    second_graph.iloc[i,i] = 0
second_graph = second_graph.fillna(0)

# Interaction Graph

In [ ]:
%%time
interaction_graph = (
    df_raw[["startstationid", "endstationid"]]
    .groupby(["startstationid", "endstationid"])
    .size()
    .reset_index(name="counts")
)
interaction_graph = interaction_graph.pivot_table(
    index="startstationid", columns="endstationid", values="counts", fill_value=0
)

# caculate station alive size
alive_size = alive_df.query('time >= @train_period[0] & time < @train_period[1] & is_alive == 1').groupby(['stationid'], as_index=False).size()
alive_size = alive_size.pivot_table(
    index="stationid", columns="stationid", values="size", fill_value=0
)
alive_size = np.power(alive_size.values.astype(np.float32), -1)
alive_size = np.where(np.isinf(alive_size), 0, alive_size)
    
# calulate mean hour interaction
for i in range(len(interaction_graph)):
    interaction_graph.iloc[i,i] = 0
interaction_graph_out = pd.DataFrame(np.matmul(alive_size, interaction_graph.values), index=interaction_graph.index, columns=interaction_graph.columns)
interaction_graph_in = pd.DataFrame(np.matmul(interaction_graph.values, alive_size), index=interaction_graph.index, columns=interaction_graph.columns)

# Correlation Graph

In [ ]:
correlation_graph = df_raw[["endstationid", "startstationid", "stoptime", "starttime"]]
correlation_graph = correlation_graph.assign(
    stoptime=correlation_graph.stoptime.dt.floor(predict_time),
    starttime=correlation_graph.starttime.dt.floor(predict_time),
)

In [ ]:
%%time
correlation_graph_in = (
    correlation_graph.groupby(["endstationid", "stoptime"])
    .size()
    .reset_index(name="flow_in")
)
correlation_graph_in = correlation_graph_in.pivot_table(
    index="stoptime", columns="endstationid", values="flow_in", fill_value=0
)
correlation_graph_in = correlation_graph_in.corr("pearson")
for i in range(len(correlation_graph_in)):
    correlation_graph_in.iloc[i,i] = 0
correlation_graph_in_positive = correlation_graph_in.where(correlation_graph_in > 0, 0)
correlation_graph_in_negative = correlation_graph_in.where(correlation_graph_in < 0, 0).abs()

In [ ]:
%%time
correlation_graph_out = (
    correlation_graph.groupby(["startstationid", "starttime"])
    .size()
    .reset_index(name="flow_out")
)
correlation_graph_out = correlation_graph_out.pivot_table(
    index="starttime", columns="startstationid", values="flow_out", fill_value=0
)
correlation_graph_out = correlation_graph_out.corr("pearson")
for i in range(len(correlation_graph_out)):
    correlation_graph_out.iloc[i,i] = 0
correlation_graph_out_positive = correlation_graph_out.where(correlation_graph_out > 0, 0)
correlation_graph_out_negative = correlation_graph_out.where(correlation_graph_out < 0, 0).abs()

# Edge Index

In [ ]:
def graph_norm(graph, axis):
    deg = np.array(np.sum(graph, axis=axis))
    deg = np.matrix(np.diag(deg)).astype(np.float32)
    deg_inv = np.power(deg,-1)
    deg_inv = np.where(np.isinf(deg_inv), 0, deg_inv)
    A_norm = np.matmul(deg_inv, graph) + np.identity(graph.shape[0])
    A_norm = torch.tensor(A_norm, dtype=torch.float32)
    
    return A_norm

In [ ]:
graph_out = torch.stack([graph_norm(graph.values, 1) for graph in [distance_graph, second_graph, interaction_graph_out, correlation_graph_in_positive, correlation_graph_in_negative, correlation_graph_out_positive, correlation_graph_out_negative]])
graph_in = torch.stack([graph_norm(graph.values, 0) for graph in [distance_graph, second_graph, interaction_graph_in, correlation_graph_in_positive, correlation_graph_in_negative, correlation_graph_out_positive, correlation_graph_out_negative]])

# Split Training Validation Testing Set

In [ ]:
station_num = len(station_info)
train_len = ((features.time >= train_period[0]) & (features.time < train_period[1])).sum() // station_num
val_len = ((features.time >= valid_period[0]) & (features.time < valid_period[1])).sum() // station_num
test_len = ((features.time >= test_period[0]) & (features.time < test_period[1])).sum() // station_num
print(f"train length:{train_len}\nvalidation length:{val_len}\ntest length:{test_len}")

In [ ]:
columns = features.columns
columns.to_list()

In [ ]:
gcn_column = columns[columns.str.contains('|'.join(['bike', 'flow', 'gender', 'usertype', 'Hourly']))]
print(len(gcn_column))
gcn_column

In [ ]:
fc_column = columns[~columns.str.contains('|'.join(['flow','bike','stationid', 'time', "y_in", "y_out"]))]
print(len(fc_column))
fc_column

In [ ]:
y_column = ["y_in", "y_out"]
y_column

In [ ]:
training_loader = []
for i in tqdm(np.arange(1, train_len + 1, 1)):
    x = torch.tensor(
        features.iloc[(i - 1) * station_num : i * station_num][gcn_column].values,
        dtype=torch.float,
    ).unsqueeze(0)
    y = torch.tensor(
        features.iloc[(i - 1) * station_num : i * station_num][y_column].values,
        dtype=torch.float,
    )
    x_fc = torch.tensor(
        features.iloc[(i - 1) * station_num : i * station_num][fc_column].values,
        dtype=torch.float,
    )
    is_alive = torch.tensor(
        alive_df.iloc[(i - 1) * station_num : i * station_num].is_alive.values,
        dtype=torch.int8,
    )
    training_loader.append(
        Data(
            x=x,
            y=y,
            x_fc=x_fc,
            is_alive=is_alive,
        )
    )

In [ ]:
validation_loader = []
for i in tqdm(np.arange(train_len + 1, train_len + val_len + 1, 1)):
    x = torch.tensor(
        features.iloc[(i - 1) * station_num : i * station_num][gcn_column].values,
        dtype=torch.float,
    ).unsqueeze(0)
    y = torch.tensor(
        features.iloc[(i - 1) * station_num : i * station_num][y_column].values,
        dtype=torch.float,
    )
    x_fc = torch.tensor(
        features.iloc[(i - 1) * station_num : i * station_num][fc_column].values,
        dtype=torch.float,
    )
    is_alive = torch.tensor(
        alive_df.iloc[(i - 1) * station_num : i * station_num].is_alive.values,
        dtype=torch.int8,
    )
    validation_loader.append(
        Data(
            x=x,
            y=y,
            x_fc=x_fc,
            is_alive=is_alive,
        )
    )

In [ ]:
testing_loader = []
for i in tqdm(
    np.arange(train_len + val_len + 1, train_len + val_len + test_len + 1, 1)
):
    x = torch.tensor(
        features.iloc[(i - 1) * station_num : i * station_num][gcn_column].values,
        dtype=torch.float,
    ).unsqueeze(0)
    
    y = torch.tensor(
        features.iloc[(i - 1) * station_num : i * station_num][y_column].values,
        dtype=torch.float,
    )
    x_fc = torch.tensor(
        features.iloc[(i - 1) * station_num : i * station_num][fc_column].values,
        dtype=torch.float,
    )
    is_alive = torch.tensor(
        alive_df.iloc[(i - 1) * station_num : i * station_num].is_alive.values,
        dtype=torch.int8,
    )
    testing_loader.append(
        Data(
            x=x,
            y=y,
            x_fc=x_fc,
            is_alive=is_alive,
        )
    )

In [ ]:
joblib.dump(
    {
        'training_loader':training_loader,
        'validation_loader':validation_loader,
        'testing_loader':testing_loader,
        'graph_in':graph_in,
        'graph_out':graph_out,
    },
    'process_data/loader_lsgnn_simple.pt'
)